# Name:Shirui Chen ID:6958613091

In [1]:
import json
import re
import numpy as np
import pandas as pd
import itertools

### Read in files. 

In the IR dataset, we have three files in total: a query file (Query.json), a document file (Docs.json), and a ground-truth file storing the real relevance between queries and documents.

Query is formated as {queryID: the content of a query}

Document is formated as {docID: the content of a document}

In both cases, the content is represented as a string.

You can use the read_file() function below to read queries and documents first. The details of the ground-truth file will be provided later, when we use it for evaluation.

As in previous homeworks, you are free to define helper functions and global variables of your own, as you find them necessary. However, the signature (inputs and outputs) of the functions that we provide must not be changed.

In [2]:
def read_file(filepath):
    # read file
    """
    Input: pathname
    Output: dictionary-format file
    
    """
    f = open(filepath, 'r')
    file = json.load(f)
    return file

In [3]:
Query = read_file('/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW4/IR_dataset/Query.json')
Document = read_file('/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW4/IR_dataset/Docs.json')

# Q1: Preprocessing
---

(20 points) Pre-process the text (which is a string) in the query and document data structures read in above. You need to segment the text into word tokens and replace the upper case in words with lower case. 

Note that this function will be used to preprocess both the queries and documents above. 

*Input*: file_dict (document-based structure output by read_file function, i.e., Queries, Docs)

*Output*: Seg_Docs (dict)

Each key in Seg_Docs corresponds to either a queryID or docID, based on what was input. Instead of being a string, however, the values corresponding to the key should be a set of ‘clean’ words corresponding to the preprocessed string. The example will help make this clearer:

Suppose there is a hypothetical entry in Queries: {123: "What's your favorite food?"} and you preprocessed "What's your favorite food?" as [“what’s”, “your”, “favorite”, “food”]. Seg_Docs will then contain {123: [“what’s”, “your”, “favorite”, “food”]}. Note that this is an example only: you must follow the preprocessing guidelines we provided at the beginning of this description. 

---

### Extra credit: 
(10 points) Some other processing operations that are highly recommended and will earn you extra credit:  removing punctuations, removing stopwords, only focusing on English words. We will be flexible in how you interpret these, as long as you are consistent. If you decide to implement these, make sure to add a comment below saying what 'extra' preprocessing steps you are implementing. 


I remove the punctuations, numbers, and stopwords.

In [4]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def segmentation(file_dict):
    Seg_Docs = {key.lower(): list(map(str, value.lower().split())) for key, value in file_dict.items()}
    Seg_Docs = {k: [w for w in l if w not in ENGLISH_STOP_WORDS and w.isalpha()] for k, l in Seg_Docs.items()}
    return Seg_Docs

In [5]:
q = segmentation(Query)
len(q)

50

In [6]:
q

{'915593': ['types', 'food', 'cook', 'sous', 'vide'],
 '794429': ['sculpture', 'shape', 'space'],
 '801118': ['supplemental', 'security', 'income', 'used'],
 '190044': ['foods', 'detox', 'liver', 'naturally'],
 '451602': ['definition', 'mechanical', 'ventilation'],
 '177604': ['eating', 'foods', 'considered', 'warm'],
 '966413': ['benefits', 'cinnamon'],
 '588587': ['causes', 'heavy', 'metal', 'toxins', 'body'],
 '1108939': ['slows', 'flow', 'blood'],
 '507445': ['symptoms', 'different', 'types', 'brain', 'bleeds'],
 '527433': ['types', 'dysarthria', 'cerebral', 'palsy'],
 '1056204': ['steam', 'boat', 'operator'],
 '1114646': ['famvir', 'prescribed'],
 '264403': ['long', 'recovery', 'face', 'lift', 'neck', 'lift'],
 '87452': ['causes', 'military', 'suicide'],
 '489204': ['right', 'pelvic', 'pain', 'causes'],
 '182539': ['example', 'monotonic', 'function'],
 '537817': ['vitamin', 'e', 'anti', 'scar'],
 '86606': ['causes', 'gas', 'large', 'intestine'],
 '88495': ['causes'],
 '877809': ['

In [7]:
d = segmentation(Document)
len(d)

5000


# Q1.1:   
(5 points) How many queries and documents are there in the dataset? 
50 queries and 5000 documents.

# Q1.2: 

(10 points) Write a function that takes the output of **segmentation(Docs)** \[**which is a dictionary, as noted above**\] as input, and returns a set called vocabulary that computes the vocabulary across the documents. Note that you should not be using the output of **segmentation(Queries)** in this function, as the queries would be unknown in advance if you were building a real system.  

In [8]:
def vocab(seg_Docs):
    global Vocabulary
    Vocabulary = []
    for values in seg_Docs.values():
        for v in values:
            if v not in Vocabulary:
                Vocabulary.append(v)
            else:
                continue
    return Vocabulary

In [9]:
Vocabulary = vocab(d)
Vocabulary = sorted(Vocabulary)
len(Vocabulary)

16905

# Q2: Calculate TF-IDF
---

(10 points) Finish the calculate_TF function below to get term frequencies for words in each document or query. The **input** is a **Seg_Docs dict** (corresponding either to preprocessed Queries or preprocessed Docs) and **either a queryID or a documentID** corresponding to a key in that dict. 

**Before you ‘access’ an item in Seg_Docs using the provided key, make sure to check that the key exists in Seg_Docs. If you do not do this check, we will take off 5 points.** <u>Print an error message if the check fails. </u>

We will be representing the TF of the query or document (the output) as a dictionary. The key should be a word and the value should be the term frequency of that word. 

The **output** is a **dictionary**, with words as keys and TF of words in the input list as values.

---

(10 points) Then calculate the IDF of each word by completing calculate_IDF. Here, you can use the output of the vocabulary function that you wrote for **Q1.2**. The **input** is again a **Seg_Docs dictionary**, but the second input is now the **vocabulary**, as the IDF is computed globally. The **output** should be a **dictionary** with a word in the vocabulary as key and the IDF of that word as value. Use the logarithmic IDF as defined in lecture. 

**Make sure to do a check before returning that every word in the vocabulary is in your output. Otherwise, we will take 5 points off.** <u>Print an error message if the check fails. </u>

In [10]:
def calculate_TF(Seg_Docs, ID):
    global TF
    if ID not in Seg_Docs:
        print('Error')
    else:
        Seg_Docs = Seg_Docs[ID]
        TF = {i:[Seg_Docs.count(i)/len(Seg_Docs)] for i in Seg_Docs}
        return TF

In [11]:
calculate_TF(d,'1396705')

{'cooking': [0.07142857142857142],
 'ingredients': [0.07142857142857142],
 'sous': [0.07142857142857142],
 'vide': [0.07142857142857142],
 'means': [0.07142857142857142],
 'throw': [0.07142857142857142],
 'blender': [0.07142857142857142],
 'right': [0.07142857142857142],
 'hollandaise': [0.07142857142857142],
 'come': [0.07142857142857142],
 'velvety': [0.07142857142857142],
 'add': [0.07142857142857142],
 'eggs': [0.07142857142857142],
 'pot': [0.07142857142857142]}

In [12]:
from collections import Counter
import math
def calculate_IDF(seg_Docs, vocabulary):
    N = len(seg_Docs)
    t = Counter()
    res = {}
    for i in seg_Docs.values():
        features = set(i)
        for f in features:
            if f not in t:
                t[f] = 1
            else:
                t[f] += 1
    if vocabulary not in t:
        print('Error')
    else:
        IDF = math.log(float(N) / t[vocabulary])
        res[vocabulary] = IDF
        return res

In [13]:
calculate_IDF(d,'cooking')

{'cooking': 4.605170185988092}

In [14]:
IDF = {}
for i in Vocabulary:
    x = calculate_IDF(d, i)
    for key,val in x.items():
        IDF[key] = val
print(IDF)

{'aa': 7.824046010856292, 'aaa': 8.517193191416238, 'aacs': 8.517193191416238, 'aacsb': 8.517193191416238, 'aaliyah': 8.517193191416238, 'aamc': 8.517193191416238, 'aap': 8.517193191416238, 'aaralyn': 8.517193191416238, 'aarghdvaark': 8.517193191416238, 'aaron': 7.824046010856292, 'aat': 8.517193191416238, 'ab': 8.517193191416238, 'abacus': 8.517193191416238, 'abalone': 8.517193191416238, 'abandoned': 8.517193191416238, 'abbe': 8.517193191416238, 'abbey': 8.517193191416238, 'abbreviated': 6.725433722188183, 'abbreviation': 6.571283042360924, 'abc': 7.824046010856292, 'abdomen': 4.8283137373023015, 'abdomend': 8.517193191416238, 'abdominal': 4.390058806371146, 'abdul': 8.517193191416238, 'abe': 8.517193191416238, 'aberdeen': 7.824046010856292, 'aberration': 8.517193191416238, 'abgc': 8.517193191416238, 'abiah': 8.517193191416238, 'abide': 8.517193191416238, 'abides': 8.517193191416238, 'abilities': 7.418580902748128, 'ability': 4.779523573132869, 'abiotic': 8.517193191416238, 'ablation'

# Q3: Encode text into vector
---

(30 points): Finish the function below called encode_TFIDF_Vector that takes Seg_Docs (dict) and the Vocabulary (set) as input, and returns the tf-idf matrix corresponding to either the queries or documents (depending on whether you are sending in the queries or documents Seg_Docs as input). Note, however, that the **Vocabulary** must be the **vocabulary you computed over the documents in both cases**. We cannot compute a vocabulary over the queries for the reason I mentioned earlier! In the discussion below, I will assume that you use the processed documents as input; however, the same observations will also apply to the queries. 

Assuming the vocabulary has V words inside it, and there are D documents, the tf-idf matrix will have dimensionality **D X V**. Each row corresponds to the tf-idf vector of a single document. 

You need to decide which documentID in Seg_Docs corresponds to which row in the matrix. One way to do this (and that you should use in this question) is to **first get all the keys within Seg_Docs as a list and then sort the list in ascending order**. There are easy Python functions to help you do all of this. Now that you have an ‘ordering’ over the documentIDs, you can use that to construct the matrix. For example, if the sorted list is $[d1,d2…,]$ then the first row in the matrix will be the tfidf vector of $d1$, the second row will be the tfidf vector of $d2$, and so on. 

Make sure to use the functions you implemented before wisely. 

<u>*Hint*: You may want to store the IDF dict you computed earlier as a global variable, so it is easily accessible inside the function. It is up to you whether to separately store the tf dicts you computed earlier, or to just call the calculate_TF function from inside this function. </u>


You must be able to convert the tf and idf dicts you get into proper vectors. Once again, we require you to use the sorting methodology above: convert and sort vocabulary into a list (ascending order); as you will remember from class, each word in the vocabulary corresponds to a dimension in the tfidf vector.  

A last point concerns words that you may see in the queries that are not in your vocabulary (which was computed over the documents). If you encounter such words, ignore them. For example, given a (preprocessed) query \[“what’s”, “your”, “favorite”, “ice-cream”\], suppose ice-cream is not in the vocabulary. You will not be able to compute a tf-idf score for it, and hence, you must ignore it. 

Execute get_TFIDF_Matrix twice: once for the query Seg_Docs, and the other for the document Seg_Docs. Assuming that you have Q queries and D documents (you computed this in **Q1.1**), you should get two matrices of dimensionality **Q X V** and **D X V** respectively. The variable representing these two matrices should be called **Query_TFIDF** and **Document_TFIDF** respectively. For example:

Query_TFIDF = TFIDF_Matrix(Query_Seg_Docs, Vocabulary)

In [15]:
def get_TFIDF_Matrix(Seg_Docs, Vocabulary):
    l = []
    for key in Seg_Docs.keys():
        l.append(key)
    l = sorted(l)
    matrix = np.zeros((len(l),len(Vocabulary)))
    for i in range(len(l)):
        TF = calculate_TF(Seg_Docs, l[i])
        for key,val in TF.items():
            if key in Vocabulary:
                matrix[i][Vocabulary.index(key)] = val[0]*list(IDF.values())[list(IDF.keys()).index(key)]
    return matrix

In [16]:
Document_TFIDF = get_TFIDF_Matrix(d,Vocabulary)
Document_TFIDF

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
Query_TFIDF = get_TFIDF_Matrix(q,Vocabulary)
Query_TFIDF

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
x1 = [np.count_nonzero(Query_TFIDF [i,:]) for i in range(len(Query_TFIDF))]
avg_x1 = sum(x1) / len(x1)
avg_x1

3.4

### Extra credit: 


(10 points): Define and implement def Incidence_Matrix(Seg_Docs, Vocabulary), which is similar to the tf-idf matrix, but contains incidence vectors (with only 0 and 1) instead of tf-idf vectors. Execute the function twice. The two matrices should be called Query_Inc and Document_Inc.

---
(5 points): On average, how many non-zero entries are there per row in the Query_TFIDF versus Query_Inc? 


3.4 non-zero entries per row in Query_TFIDF.
3.4 non-zero entries per row in Query_Inc.

In [19]:
def Incidence_Matrix(Seg_Docs, Vocabulary):
    l = []
    for key in Seg_Docs.keys():
        l.append(key)
    l = sorted(l)
    matrix = np.zeros((len(l),len(Vocabulary)))
    for i in range(len(l)):
        TF = calculate_TF(Seg_Docs, l[i])
        for key,val in TF.items():
            if key in Vocabulary:
                matrix[i][Vocabulary.index(key)] = 1
    return matrix

In [20]:
Document_Inc = Incidence_Matrix(d, Vocabulary)
Document_Inc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
Query_Inc = Incidence_Matrix(q, Vocabulary)
Query_Inc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
x2 = [np.count_nonzero(Query_Inc[i,:]) for i in range(len(Query_Inc))]
avg_x2 = sum(x2) / len(x2)
avg_x2

3.4

# Q4: Calculate similarity between queries and documents
---

# Q4.1:
(15 points): Now we are ready to calculate the ranked list of documents retrieved for each query. To do so, you must first calculate the similarity matrix between Query_TFIDF and Document_TFIDF. However, before we can multiply the matrices to get cosine similarity, we must normalize them. The function below should **normalize** each row-vector in **orig_matrix** in turn and **output** another **matrix** (called normalized_matrix) of identical dimensionality but where the rows are normalized. 

In [23]:
from sklearn.preprocessing import normalize
def normalize_matrix_l2_norm(orig_matrix):
    normalized_matrix = normalize(orig_matrix)
    return normalized_matrix

# Q4.2:
​
(5 points): Execute the normalize_matrix_l2_norm twice, using Query_TFIDF and Document_TFIDF as inputs respectively. To save space, store the output in the same variable e.g., Query_TFIDF = normalize_matrix_l2_norm(Query_TFIDF)
​

In [24]:
Query_TFIDF = normalize_matrix_l2_norm(Query_TFIDF)
Document_TFIDF = normalize_matrix_l2_norm(Document_TFIDF)

In [25]:
Query_TFIDF

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
Document_TFIDF

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Q4.3:
​
(20 points) Compute the ‘cosine similarity’ matrix between **Query_TFIDF** and **Document_TFIDF** by completing the function below. dot_product(A,B) returns another matrix that equals $A \cdot B^T$. You must do a check to make sure that the number of columns in A is the same as the number of rows in B. Otherwise, we will deduct 5 points. <u>Print an error message if the check fails. </u>


In [27]:
def dot_product(A, B):
    if len(A[0]) != len(B.transpose()):
        print('Error')
    else:
        return np.dot(A,B.transpose())

CS = dot_product(Query_TFIDF, Document_TFIDF)

In [28]:
CS

array([[0.        , 0.        , 0.        , ..., 0.09301875, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

<u> Hint: Your resulting matrix must have dimensionality Q X D. Each cell value represents the cosine similarity between the query represented in that row, and the document represented in that column. </u>

# Q5: Get k largest documents for each query
---

# Q5.1:
(20 points) For the questions below, feel free to write helper functions of your own. You will mainly only need the **CS matrix**, although you may need to access or store some earlier data structures that you produced as global variables.

**Print out the ranked list of top 10 most relevant document IDs for the following query IDs. If there are ties, it is okay to break them arbitrarily**:

- "794429":
- "915593":
- "527433":
- "1049519":
- "1105792":

In [29]:
def ten_most_revelant(queryID):
    lst = []
    doc_lst = list(sorted(d))
    index = list(sorted(q)).index(queryID)
    row = list(CS[index,:])
    top_10 = sorted(row)[::-1][0:10]
    for i in top_10:
        col = row.index(i)
        lst.append(doc_lst[col])
    return lst

In [30]:
ten_most_revelant("794429")

['8663243',
 '8663241',
 '6102883',
 '8663244',
 '7283249',
 '3555425',
 '3262200',
 '4933634',
 '2171822',
 '342876']

In [31]:
ten_most_revelant("915593")

['6923052',
 '275732',
 '82113',
 '5931269',
 '8103065',
 '7885529',
 '5931265',
 '82107',
 '5931270',
 '5931270']

In [32]:
ten_most_revelant("527433")

['8371764',
 '8306449',
 '269430',
 '5603298',
 '6186391',
 '6667277',
 '1379245',
 '5466807',
 '1212782',
 '8617271']

In [33]:
ten_most_revelant("1049519")

['7105',
 '2960561',
 '1083689',
 '6383244',
 '561964',
 '3525375',
 '1622751',
 '4012128',
 '2286101',
 '3542112']

In [34]:
ten_most_revelant("1105792")

['8201612',
 '8201616',
 '8201611',
 '8201613',
 '8201619',
 '8201615',
 '8201614',
 '8201617',
 '1000047',
 '1000047']

### Read in the ground-truth
​
Please use read_file function to read "QDRel.json" as the ground-truth. Each record in the ground-truth file contains a pair of QueryID and DocID, which formatted as {
        "queryID": \*\*\*,
        "docID": \*\*\*
    }. Document D1 is relevant for Query Q1 when the pair Q1 and D1 appeared in the ground-truth file.
​

<u>Note that we are assuming binary relevance here; either a document is relevant for a query or it is not.</u>
​

In [35]:
Ground_Truth = read_file('/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW4/IR_dataset/QDRel.json')

In [36]:
Ground_Truth

[{'queryID': '915593', 'docID': '1396705'},
 {'queryID': '915593', 'docID': '1907207'},
 {'queryID': '915593', 'docID': '3357360'},
 {'queryID': '915593', 'docID': '3485337'},
 {'queryID': '915593', 'docID': '4566822'},
 {'queryID': '915593', 'docID': '4910037'},
 {'queryID': '915593', 'docID': '6142152'},
 {'queryID': '915593', 'docID': '6226401'},
 {'queryID': '915593', 'docID': '7709320'},
 {'queryID': '915593', 'docID': '7861751'},
 {'queryID': '915593', 'docID': '8029524'},
 {'queryID': '915593', 'docID': '8029531'},
 {'queryID': '915593', 'docID': '8455300'},
 {'queryID': '915593', 'docID': '1396708'},
 {'queryID': '915593', 'docID': '1815965'},
 {'queryID': '915593', 'docID': '1972028'},
 {'queryID': '915593', 'docID': '210379'},
 {'queryID': '915593', 'docID': '275724'},
 {'queryID': '915593', 'docID': '275726'},
 {'queryID': '915593', 'docID': '275732'},
 {'queryID': '915593', 'docID': '2923494'},
 {'queryID': '915593', 'docID': '2923496'},
 {'queryID': '915593', 'docID': '292

# Q5.2:

(20 points): For which query among the five above does the system achieve the highest NDCG using the ground-truth? Note that we are only evaluating NDCG for the top 10 retrieved documents per query. You must use code to calculate the NDCGs (do not take this calculation 'off' the notebook). 

When queryID == "794429", it achieves the highest NDCG.

In [78]:
import math
def compute_NDCG(q_ID):
    DCG = 0
    IDCG = 0
    lst =[]
    x = ten_most_revelant(q_ID)
    for i in range(len(Ground_Truth)):
        if Ground_Truth[i]['queryID'] == q_ID:
            lst.append(Ground_Truth[i]['docID'])
    for i in range(len(lst)):
        IDCG += 1/math.log(i+2,2)
        
    new_lst = []
    for i in range(len(x)):
        for j in range(len(lst)):
            if x[i] == lst[j]:
                new_lst.append(i+2)
    
    for i in range(len(new_lst)):
        DCG += 1/math.log(new_lst[i],2)
    
    NDCG = DCG/IDCG
    return NDCG

In [79]:
ans_NDCG = []
for i in ["794429","915593","527433","1049519","1105792"]:
    ans_NDCG.append(compute_NDCG(i))
ans_NDCG

[0.9438661545147249,
 0.2303126433712897,
 0.2661100399694587,
 0.46927872602275655,
 0.776294131638404]

# Q5.3:

(20 points): Using only the five queries and the top 10 retrieved documents per query, what is the MAP of the system?

In [81]:
def compute_AP(q_ID):
    lst = []
    x = ten_most_revelant(q_ID)
    y = []
    AP = 0
    for i in range(len(Ground_Truth)):
        if Ground_Truth[i]['queryID'] == q_ID:
            lst.append(Ground_Truth[i]['docID'])
    for i in range(len(lst)):
        for j in range(len(x)):
            if x[j] in lst[i]:
                y.append((j+1)/(lst.index(x[j])+1))
    AP = sum(y)/len(y)
    return AP

In [82]:
ans_AP = []
for i in ["794429","915593","527433","1049519","1105792"]:
    ans_AP.append(compute_AP(i))
MAP = sum(ans_AP)/5
MAP

0.6037642363050553

# Q5.4:
(10 points): Is the average of the NDCG computed over the five queries significantly different from 0.3 at the 95% confidence level? Print out the p-value of your test. 

<u>Hint: Will the z-test work here? If not, which test should you be using? </u>


In [80]:
import statistics
avg_NDCG = sum(ans_NDCG)/5

0.5371723391033267 0.3141760162443578


In [120]:
from scipy import stats
stats.ttest_1samp(ans_NDCG,0.3)

Ttest_1sampResult(statistic=1.6880138686499566, pvalue=0.16668065302242013)

p = 0.167 > 0.05. We cannot reject the null hypothesis.

### Extra Credit: 

(20 points): Using the incidence matrices you computed earlier, does your answer to Q5.2 change? What is the new answer?

In [123]:
new_CS = dot_product(Query_Inc,Document_Inc)
new_CS

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [124]:
def ten_most_revelant_Inc(queryID):
    lst = []
    doc_lst = list(sorted(d))
    index = list(sorted(q)).index(queryID)
    row = list(new_CS[index,:])
    top_10 = sorted(row)[::-1][0:10]
    for i in top_10:
        col = row.index(i)
        lst.append(doc_lst[col])
    return lst

In [129]:
import math
def compute_NDCG_Inc(q_ID):
    DCG = 0
    IDCG = 0
    lst =[]
    x = ten_most_revelant_Inc(q_ID)
    for i in range(len(Ground_Truth)):
        if Ground_Truth[i]['queryID'] == q_ID:
            lst.append(Ground_Truth[i]['docID'])
    for i in range(len(lst)):
        IDCG += 1/math.log(i+2,2)
        
    new_lst = []
    for i in range(len(x)):
        for j in range(len(lst)):
            if x[i] == lst[j]:
                new_lst.append(i+2)
    
    for i in range(len(new_lst)):
        DCG += 1/math.log(new_lst[i],2)
    
    NDCG = DCG/IDCG
    return NDCG

In [131]:
ans_NDCG_Inc = []
for i in ["794429","915593","527433","1049519","1105792"]:
    ans_NDCG_Inc.append(compute_NDCG_Inc(i))
ans_NDCG_Inc

[0.8318724637288826,
 0.2303126433712897,
 0.2661100399694587,
 0.46927872602275655,
 0.776294131638404]